In [1]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617

In [2]:
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras import Model
import numpy as np

In [3]:
inputs = Input(shape=(5,5,3))
x = Conv2D(2, 3)(inputs)
model = Model(inputs, x)

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5, 5, 3)]         0         
                                                                 
 conv2d (Conv2D)             (None, 3, 3, 2)           56        
                                                                 
Total params: 56
Trainable params: 56
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.weights

[<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 3, 2) dtype=float32, numpy=
 array([[[[-0.2214439 , -0.15195417],
          [-0.33063164, -0.2934303 ],
          [ 0.05383286, -0.29145738]],
 
         [[-0.03459874, -0.18074483],
          [ 0.0829632 , -0.18525888],
          [-0.30195278,  0.28627008]],
 
         [[-0.16308369, -0.00504276],
          [ 0.3425789 ,  0.23431945],
          [ 0.3600672 , -0.12323308]]],
 
 
        [[[-0.32080007,  0.34273505],
          [ 0.18235081,  0.10385716],
          [-0.1344554 , -0.33093956]],
 
         [[ 0.26645142,  0.13327193],
          [ 0.14260197, -0.01873457],
          [ 0.04164705,  0.09329805]],
 
         [[ 0.22617143,  0.00685191],
          [ 0.04963994,  0.23333359],
          [ 0.35556698, -0.2943074 ]]],
 
 
        [[[-0.01687273, -0.23511177],
          [ 0.3297963 ,  0.34760195],
          [ 0.03496316,  0.09493944]],
 
         [[ 0.15188134, -0.36046585],
          [ 0.03922135,  0.09752569],
          [-0.07491425,  0

In [6]:
X = np.random.rand(1,5,5,3)
X

array([[[[0.01897312, 0.08743388, 0.86222637],
         [0.10109003, 0.28028469, 0.47527166],
         [0.01512083, 0.32085385, 0.55800198],
         [0.4642667 , 0.30473978, 0.4908111 ],
         [0.35508257, 0.37433134, 0.78504401]],

        [[0.86741939, 0.20455078, 0.75628909],
         [0.49125544, 0.77870243, 0.00276785],
         [0.15663838, 0.57212579, 0.99557935],
         [0.94449608, 0.24962475, 0.12299833],
         [0.99733666, 0.01757025, 0.78783541]],

        [[0.48878705, 0.12012174, 0.9315336 ],
         [0.21785634, 0.36211795, 0.66387035],
         [0.90845156, 0.08354312, 0.49792257],
         [0.89275096, 0.48364178, 0.29199137],
         [0.42604607, 0.48339958, 0.24699135]],

        [[0.49614912, 0.82873805, 0.51174584],
         [0.40336063, 0.91275723, 0.98636519],
         [0.77062345, 0.9014581 , 0.37892572],
         [0.70837991, 0.13523834, 0.27672346],
         [0.80903352, 0.01406841, 0.65278234]],

        [[0.04710218, 0.17104368, 0.60839461],
     

In [7]:
y = model.predict(X)
y

1/1 [==============================] - 0s 31ms/step


2023-10-23 20:04:35.358347: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


array([[[[ 0.7669913 ,  0.15274508],
         [ 0.7217935 ,  0.43549562],
         [ 0.956968  , -0.7293344 ]],

        [[ 0.82161874, -0.08652828],
         [ 0.45854256,  0.37499383],
         [ 0.6369549 , -0.12588985]],

        [[ 0.3617655 ,  0.5265081 ],
         [ 0.5842541 ,  0.11949164],
         [ 0.6298134 , -0.148645  ]]]], dtype=float32)

In [8]:
X_in = [[[int(X[0][i][j][k]*1e36) for k in range(3)] for j in range(5)] for i in range(5)]
weights = [[[[int(model.weights[0].numpy()[i][j][k][l]*1e36) for l in range(2)] for k in range(3)] for j in range(3)] for i in range(3)]
bias = [int(model.weights[1].numpy()[i]*1e72) for i in range(2)]

In [9]:
def Conv2DInt(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    Input = [[[str(input[i][j][k] % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    Weights = [[[[str(weights[i][j][k][l] % p) for l in range(nFilters)] for k in range(nChannels)] for j in range(kernelSize)] for i in range(kernelSize)]
    Bias = [str(bias[i] % p) for i in range(nFilters)]
    out = [[[0 for _ in range(nFilters)] for _ in range((nCols - kernelSize)//strides + 1)] for _ in range((nRows - kernelSize)//strides + 1)]
    remainder = [[[None for _ in range(nFilters)] for _ in range((nCols - kernelSize)//strides + 1)] for _ in range((nRows - kernelSize)//strides + 1)]
    for i in range((nRows - kernelSize)//strides + 1):
        for j in range((nCols - kernelSize)//strides + 1):
            for m in range(nFilters):
                for k in range(nChannels):
                    for x in range(kernelSize):
                        for y in range(kernelSize):
                            out[i][j][m] += input[i*strides+x][j*strides+y][k] * weights[x][y][k][m]
                out[i][j][m] += bias[m]
                remainder[i][j][m] = str(out[i][j][m] % n)
                out[i][j][m] = str(out[i][j][m] // n % p)
    return Input, Weights, Bias, out, remainder

In [10]:
X_in, weights, bias, out, remainder = Conv2DInt(5, 5, 3, 2, 3, 1, 10**36, X_in, weights, bias)
out, remainder

([[['766991308610348168697678317289253491',
    '152745147346569443334783280814065046'],
   ['721793514622685877535765261453578285',
    '435495635689848419191313701414477907'],
   ['956968022633247456612037533749279884',
    '21888242871839275222246405745257275088547635065957648765434334612742599273530']],
  [['821618728955717473031173417522483642',
    '21888242871839275222246405745257275088548277872099372923381843387091765217066'],
   ['458542547820893552234319995890476448',
    '374993810980093272528286581550476085'],
   ['636954799020811143676097566965150012',
    '21888242871839275222246405745257275088548238510569604144298779927709876426450']],
  [['361765493481699222011118768701693995',
    '526508029475490175970042650793369367'],
   ['584254113344261709635215513635894572',
    '119491582294697734028918864863442650'],
   ['629813449657849355217882687452958040',
    '21888242871839275222246405745257275088548215755396606066460162924422748870356']]],
 [[['91738091344179164138305719

In [11]:
in_json = {
    "in": X_in,
    "weights": weights,
    "bias": bias,
    "out": out,
    "remainder": remainder
}

In [12]:
import json

In [13]:
with open("conv2D_input.json", "w") as f:
    json.dump(in_json, f)

In [14]:
inputs = Input(shape=(10,10,3))
x = Conv2D(2, 4, 3)(inputs)
model = Model(inputs, x)

In [15]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10, 10, 3)]       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 3, 2)           98        
                                                                 
Total params: 98
Trainable params: 98
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.weights

[<tf.Variable 'conv2d_1/kernel:0' shape=(4, 4, 3, 2) dtype=float32, numpy=
 array([[[[-0.2644725 , -0.06314689],
          [-0.19228128, -0.15511811],
          [ 0.11695373,  0.15373573]],
 
         [[-0.25127184, -0.03327389],
          [ 0.06582499,  0.13954943],
          [-0.02274385, -0.24024239]],
 
         [[-0.27345484,  0.07233012],
          [ 0.03240535, -0.1362924 ],
          [ 0.16554734, -0.19980597]],
 
         [[ 0.12271923, -0.16348948],
          [-0.24399345,  0.11112538],
          [ 0.04153693,  0.09944585]]],
 
 
        [[[ 0.07061589,  0.08067289],
          [-0.25938636, -0.04698843],
          [-0.18615322, -0.18926641]],
 
         [[ 0.06828818,  0.10024589],
          [-0.06301631, -0.21391404],
          [ 0.17798159,  0.16077739]],
 
         [[-0.18883933, -0.24141382],
          [-0.1410877 , -0.1870578 ],
          [-0.17638391,  0.12909776]],
 
         [[ 0.24995095, -0.27214956],
          [-0.16633804, -0.24534652],
          [ 0.09470761, -0.

In [17]:
X = np.random.rand(1,10,10,3)
X

array([[[[0.74916437, 0.48058547, 0.76332072],
         [0.12424725, 0.17444765, 0.23397784],
         [0.05504572, 0.73761126, 0.8872156 ],
         [0.49900099, 0.92746041, 0.53705474],
         [0.55972545, 0.16210532, 0.48132197],
         [0.5848511 , 0.49854495, 0.08101739],
         [0.18151106, 0.19916602, 0.72216404],
         [0.8740212 , 0.76936566, 0.77574156],
         [0.640459  , 0.97982307, 0.28733369],
         [0.72894846, 0.08559827, 0.88796122]],

        [[0.72655621, 0.74204472, 0.22625834],
         [0.04455912, 0.02637621, 0.7410693 ],
         [0.59282978, 0.27570664, 0.76839831],
         [0.98853958, 0.32472733, 0.07214331],
         [0.18975449, 0.50827871, 0.61454778],
         [0.04411632, 0.5425321 , 0.08095671],
         [0.98276969, 0.93905031, 0.27580299],
         [0.01991902, 0.18288148, 0.56430848],
         [0.60873785, 0.76133969, 0.94420434],
         [0.29177495, 0.60971797, 0.75330394]],

        [[0.73958658, 0.86792802, 0.38199042],
         

In [18]:
y = model.predict(X)
y

1/1 [==============================] - 0s 22ms/step


array([[[[-0.02103192, -0.00251862],
         [-0.13590473, -0.43280643],
         [-0.46592993, -0.6076722 ]],

        [[-0.41985548, -0.28316095],
         [-0.5124401 , -0.67603445],
         [-0.30362517, -0.35066307]],

        [[ 0.04719239, -0.77217025],
         [-0.90729254, -0.5460546 ],
         [-0.6157329 , -0.56018454]]]], dtype=float32)

In [19]:
X_in = [[[int(X[0][i][j][k]*1e36) for k in range(3)] for j in range(10)] for i in range(10)]
weights = [[[[int(model.weights[0].numpy()[i][j][k][l]*1e36) for l in range(2)] for k in range(3)] for j in range(4)] for i in range(4)]
bias = [int(model.weights[1].numpy()[i]*1e72) for i in range(2)]

In [20]:
X_in, weights, bias, out, remainder = Conv2DInt(10, 10, 3, 2, 4, 3, 10**36, X_in, weights, bias)
out, remainder

([[['21888242871839275222246405745257275088548343368429240512769054624061005959510',
    '21888242871839275222246405745257275088548361881856976212445268734113123308000'],
   ['21888242871839275222246405745257275088548228495669861064087547662558271715284',
    '21888242871839275222246405745257275088547931594040764149614699120473995978124'],
   ['21888242871839275222246405745257275088547898470480336398546468994244949638791',
    '21888242871839275222246405745257275088547756728190784802798484857861334105370']],
  [['21888242871839275222246405745257275088547944544909737422432087456874316949324',
    '21888242871839275222246405745257275088548081239515185091772880603232052577114'],
   ['21888242871839275222246405745257275088547851960358367500079371782793385911358',
    '21888242871839275222246405745257275088547688365801627855012444638213091512422'],
   ['21888242871839275222246405745257275088548060775297780413313747663983226225838',
    '218882428718392752222464057452572750885480137372493022

In [21]:
in_json = {
    "in": X_in,
    "weights": weights,
    "bias": bias,
    "out": out,
    "remainder": remainder
}

In [22]:
with open("conv2D_stride_input.json", "w") as f:
    json.dump(in_json, f)